# Apercal tutorial 0: Introduction

The first step is to initialise APERCAL. You need your `apercal` directory added to your `$PYTHONPATH` before you can use it.

In [1]:
import os
import sys

In [2]:
here = os.path.abspath(os.curdir)
root = os.path.abspath(here + '/../..')
sys.path.append(root)
config = here + '/cfg/0.cfg'

Some more setup work to do:

In [3]:
# Enable auto-completion in the notebook. The different available functions in
# the classes and their attributes should now appear while typing commands.
%config IPCompleter.greedy=True

# Import the lib module to enable the logger.
import libs.lib as lib

Setup logger with lib.setup_logger()


The log file should point to the directory you are reducing the data in.
So change this path to your apercal distribution.

In [4]:
_ = lib.setup_logger('info', logfile=here + '/logs/0.log')

root - INFO : Logging started!
root - INFO : To see the log in a bash window use the following command:
root - INFO : tail -n +1 -f /home/gijs/Work/apercal/ipython-notebooks/tutorials/logs/0.log


Now we start APERCAL. If you start the notebook a popup window will come up where you can define the IP-adress the server runs on.
Default should be ok, but be sure to disable the token security or copy the token over from the notebook server you started.

In [5]:
import apercal

In the normal mode APERCAL runs through a defined number of reduction steps with parameters given in a config file.
Reduction steps are usually executed in the following order: PREPARE, PREFLAG, CONVERT, CROSSCAL, SELFCAL, CONTINUUM, LINE, POLARISATION, MOSAIC, TRANSFER. <br>
If you want to execute a step you need to initialise the class in combination with the config-file.

In [6]:
prepare = apercal.prepare(config)

PREPARE - INFO : ### Configuration file /home/gijs/Work/apercal/ipython-notebooks/tutorials/cfg/0.cfg successfully read! ###


In [7]:
prepare.apercaldir = apercal_root
prepare.basedir = apercal_root

NameError: name 'apercal_root' is not defined

You should change the `cfg` file to point to your own copy of `apercal`, not those of Björn.

In [ ]:
if 'adebahr' in prepare.basedir or 'adebahr' in prepare.apercaldir:
    print("Please edit tutorials/cfg/0.cfg and remove the references to adebahr.")

You can look at the current parameters for this step using the show function:

In [ ]:
prepare.show()

In case you do not give a configuration file the `default.cfg` is used from the main APERCAL directory. These values should also be the ones for which the standard APERTIF data reduction works.

In [ ]:
default_prepare = apercal.prepare();

You can look at the parameters of the single step again.

In [ ]:
prepare.show()

The variables pointing to the directories for the location of the original datasets are empty now since they will be different for each individual observation.

Or look at all current parameters at once using the `showall=True` option. The explanation of all parameters can be found in the `/docs` subfolder of the APERCAL distribution.

In [ ]:
prepare.show(showall=True)

The PREPARE step creates the neccessary directories to start the reduction and copies over the datasets from the original directories to the basedir you give.
It has three options single_element, multi_element, and multi_element_90. This defines what kind of dataset you have (or want to reduce) single_element will only copy the central beam with full bandwidth, multi_element will look for all 37 beams, and multi_element_90 splits the recoreded 90 MHz band of the old limited 90 MHz observations

To execute a single step with the set parameters you can just give it a go.

In [ ]:
from time import time

In [ ]:
prepare.go()

Now let's load the convert module and see what it can do. Again exchange the location of the config file with your modified one. (We are skipping the PREFLAG step here)

In [ ]:
convert = apercal.convert(config)

You can get a basic explanation and list of methods by using the usual python `help` function. This basically gives you the comments in the code.

If you look at the go method you can check in which order the different methods (substeps) are executed.
The go method is a fast way to execute a whole step with the current settings. The functions `director`, `reset`, and `show` are available in all modules and implemented in the same way.

In [ ]:
help(convert)

It does not mean that a certain task is executed when a certain function is executed. The switch for executing a task is in the function itself to give the code a better structure.
To illustrate this let's convert the data first:

In [ ]:
convert.go()

If you look at the convert step of the config file you see that the parameters `convert_ms2uvfits` and `convert_uvfits2mir` are both set to `True`.

In [ ]:
convert.show()

Now let's execute the individual substeps of this function individually. First reset the pipeline to the state before the conversion by using the `reset` method.
This will delete all converted files in the crosscal directory. Be careful with this command. You might need to execute previous steps of the pipeline again before you can redo things.

In [ ]:
convert.reset()

The first substep of the go method is `ms2uvfits`. So let's execute only this step.
Usually the naming of the individual steps and their corresponding functions is the same.

In [ ]:
convert.ms2uvfits()

The data is now converted to UVFITS format. Now let's convert only the target data to MIRIAD format. For this we need to change the `convert_fluxcal` parameter to `False`.

In [ ]:
convert.convert_fluxcal = False

You can always change parameters while executing the pipeline. These parameters are only temporarily changed and overwritten as soon as you load a new config file or restart apercal.

In [ ]:
convert.show()

And now execute the conversion from UVFITS to MIRIAD format (only for the target data).

In [ ]:
convert.uvfits2miriad()

The naming of the data files is always dependent on the input MS-file. The renamiing in the different steps is automatically handled by the pipeline.
Even if you start with a already cross-calibrated dataset, use `<filename>.MS` for the initial step, while your actual cross-calibrated MIRIAD file is located in the crosscal subdirectory and named `<filename>.mir`
The full directory structure is fixed and documented in "`2 - Directory structure`" in the `/docs` subfolder of the APERCAL pipeline directory.

Let's reset the convert step again:

In [ ]:
convert.reset()

We can also skip the second step (uvfits2miriad) by setting the parameter to false

In [ ]:
convert.convert_uvfits2mir = False
convert.show()

Now execute with `go`:

In [ ]:
convert.go()

Now only the target data was converted and only from MS to UVFITS since the switch for converting from UVFITS to MIRIAD (`convert_uvfits2mir`) is located inside the function itself.

You can execute single MIRIAD tasks within the notebook. First you have to initialise the task using the `lib` module:

In [ ]:
fits = lib.miriad('fits')

Let's convert the NGC4062 data from UVFITS to MIRIAD 'by hand'. You need to change the directory paths to your own directories.

The MIRIAD `in` keyword needs an additional underscore at the end since python has a predefined global function `in`.

In [ ]:
fits.in_ = '/data/{}/apertif/tutorials/T0/00/crosscal/M51.UVFITS'.format(myusername)
fits.op = 'uvin'
fits.out = '/data/{}/apertif/tutorials/T0/00/crosscal/M51.mir'.format(myusername)

fits.go()